In [103]:
! pip install pillow
! pip install numpy
! pip install matplotlib
! pip install scipy

In [10]:
from PIL import Image
import numpy as np
cfa = np.array(Image.open('CFA.bmp'))
rgb_cfa = np.array(Image.open('RGB_CFA.bmp'))

cfa.shape, rgb_cfa.shape

((2073, 4176), (2073, 4176, 3))

### VNG

#### Шаг 1. Получим матрицу цветов

В каждой ячейке - номер цвета, для которого нам известна интенсивность

In [51]:
R = 0
G = 1
B = 2

In [52]:
def get_color(rgb: np.array):
    not_zero = np.where(rgb != 0)[0]
    if not_zero.size != 1:
        raise ValueError('Multiple non-zero channels in rgb_cfa. Unable to detect pixel color')
    return not_zero[0]

In [53]:
BAYER_FILTER = np.apply_along_axis(get_color, -1, rgb_cfa[:2, :2])
BAYER_FILTER

array([[0, 1],
       [1, 2]], dtype=int64)

In [54]:
import math

cfa_x, cfa_y = cfa.shape
bayer_x, bayer_y = BAYER_FILTER.shape
reps = math.ceil(cfa_x / bayer_x), math.ceil(cfa_y / bayer_y)
reps

(1037, 2088)

In [55]:
color_map = np.tile(BAYER_FILTER, reps)[:cfa_x, :cfa_y]
print(color_map, colors.shape)

[[0 1 0 ... 1 0 1]
 [1 2 1 ... 2 1 2]
 [0 1 0 ... 1 0 1]
 ...
 [0 1 0 ... 1 0 1]
 [1 2 1 ... 2 1 2]
 [0 1 0 ... 1 0 1]] (2073, 4176)


#### Шаг 2. Посчитаем градиенты и значения цветов по всем направлениям

In [104]:
a = np.array([[1,1,1,1,1], [1,1,1,1,1],[1,1,1,1,1],[1,1,1,1,1],[1,1,1,1,1]])
b = np.array([[1,1,1], [1,1,1], [1,1,1]])

In [107]:
from scipy import signal
signal.convolve2d(a, b, mode='valid')

array([[9, 9, 9],
       [9, 9, 9],
       [9, 9, 9]])

In [116]:
a = np.array([[1,2,3,4], [5,6,7,8], [9,10,11,12], [13,14,15,16]])
padded = np.pad(a, ((3,3), (3,3)), 'constant', constant_values=0)
a, padded

(array([[ 1,  2,  3,  4],
        [ 5,  6,  7,  8],
        [ 9, 10, 11, 12],
        [13, 14, 15, 16]]),
 array([[ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [ 0,  0,  0,  1,  2,  3,  4,  0,  0,  0],
        [ 0,  0,  0,  5,  6,  7,  8,  0,  0,  0],
        [ 0,  0,  0,  9, 10, 11, 12,  0,  0,  0],
        [ 0,  0,  0, 13, 14, 15, 16,  0,  0,  0],
        [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0]]))

In [125]:
a - padded[5:-1, 2:-4]

array([[ 1, -7, -7, -7],
       [ 5, -7, -7, -7],
       [ 9, 10, 11, 12],
       [13, 14, 15, 16]])

In [130]:
def calc_directions_and_grads(cfa: np.array, color_map: np.array):
    # pad cfa and color_map with zeros
    padded_cfa = np.pad(cfa, ((2, 2), (2, 2)), 'constant', constant_values=0)
    padded_3_cfa = np.pad(cfa, ((3, 3), (3, 3)), 'constant', constant_values=0)
    padded_color_map = np.pad(color_map, ((2,2), (2,2)), 'constant', constant_values=0)
    
    # <DIRECTION>_1 - in <DIRECTION> with step 1
    # <DIRECTION>_2 - in <DIRECTION> with step 2
    # <DIRECTION1>_<DIRECTION2> - in <DIRECTION1> with step1, then in <DIRECTION2> with step 1
    N_1 = padded_cfa[1:-3, 2:-2]
    N_2 = padded_cfa[:-4, 2:-2]
    E_1 = padded_cfa[2:-2, 3:-1]
    E_2 = padded_cfa[2:-2, 4:]
    S_1 = padded_cfa[3:-1, 2:-2]
    S_2 = padded_cfa[4:, 2:-2]
    W_1 = padded_cfa[2:-2, 1:-3]
    W_2 = padded_cfa[2:-2, :-4]
    NE_1 = padded_cfa[1:-3, 3:-1]
    NE_2 = padded_cfa[:-4, 4:]
    NE_N = padded_3_cfa[1:-5, 4:-2]
    NE_E = padded_3_cfa[2:-4, 5:-1]
    SE_1 = padded_cfa[3:-1, 3:-1]
    SE_2 = padded_cfa[4:, 4:]
    SE_E = padded_3_cfa[4:-2, 5:-1]
    SE_S = padded_3_cfa[5:-1, 4:-2]
    NW_1 = padded_cfa[1:-3, 1:-3]
    NW_2 = padded_cfa[:-4, :-4]
    NW_N = padded_3_cfa[1:-5, 2:-4]
    NW_W = padded_3_cfa[2:-4, 1:-5]
    SW_1 = padded_cfa[3:-1, 1:-3]
    SW_2 = padded_cfa[4:, :-4]
    SW_W = padded_3_cfa[4:-2, 1:-5]
    SW_S = padded_3_cfa[5:-1, 2:-4]
    
    cmap_N_1 = padded_color_map[1:-3, 2:-2]
    cmap_E_1 = padded_color_map[2:-2, 3:-1]
    cmap_S_1 = padded_color_map[3:-1, 2:-2]
    cmap_W_1 = padded_color_map[2:-2, 1:-3]
    
    #calc directions
    G_N = np.where(color_map == G, (cfa + N_2) / 2, N_1)
    G_E = np.where(color_map == G, (cfa + E_2) / 2, E_1)
    G_S = np.where(color_map == G, (cfa + S_2) / 2, S_1)
    G_W = np.where(color_map == G, (cfa + W_2) / 2, W_1)
    G_NE = np.where(color_map == G, NE_1, (NE_N + NE_E + N_1 + E_1) / 4) 
    G_SE = np.where(color_map == G, SE_1, (SE_E + SE_S + E_1 + S_1) / 4)
    G_NW = np.where(color_map == G, NW_1, (NW_N + NW_W + N_1 + W_1) / 4)
    G_SW = np.where(color_map == G, SW_1, (SW_W + SW_S + S_1 + W_1) / 4)
    
    R_N = np.where(color_map == R, (cfa + N_2) / 2, 
                   np.where(color_map == B, (NE_1 + NW_1) / 2, 
                       np.where(cmap_N_1 == R, N_1, (NE_N + NW_N + E_1 + W_1) / 4)))
    R_E = np.where(color_map == R, (cfa + E_2) / 2, 
                  np.where(color_map == B, (NE_1 + SE_1) / 2,
                       np.where(cmap_E_1 == R, E_1, (NE_E + SE_E + N_1 + S_1) / 4))) 
    R_S = np.where(color_map == R, (cfa + S_2) / 2,
                  np.where(color_map == B, (SE_1 + SW_1) / 2,
                       np.where(cmap_S_1 == R, S_1, (SE_S + SW_S + E_1 + W_1) / 4)))
    R_W = np.where(color_map == R, (cfa + W_2) / 2,
                  np.where(color_map == B, (NW_1 + SW_1) / 2, 
                       np.where(cmap_W_1 == R, W_1, (NW_W + SW_W + N_1 + S_1) / 4)))
    R_NE = np.where(color_map == R, (cfa + N_2 + E_2 + NE_2) / 4,
                   np.where(color_map == B, NE_1,
                       np.where(cmap_E_1 == R, (E_1 + NE_N) / 2, (N_1 + NE_E) / 2)))
    R_SE = np.where(color_map == R, (cfa + E_2 + S_2 + SE_2) / 4,
                   np.where(color_map == B, SE_1,
                       np.where(cmap_E_1 == R, (E_1 + SE_S) / 2, (S_1 + SE_E) / 2)))
    R_NW = np.where(color_map == R, (cfa + N_2 + W_2 + NW_2) / 4,
                   np.where(color_map == B, NW_1,
                       np.where(cmap_W_1 == R, (W_1 + NW_N) / 2, (N_1 + NW_W) / 2)))
    R_SW = np.where(color_map == R, (cfa + S_2 + W_2 + SW_2) / 4,
                   np.where(color_map == B, SW_1,
                       np.where(cmap_W_1 == R, (W_1 + SW_S) / 2, (S_1 + SW_W) / 2)))
    
    B_N = np.where(color_map == B, (cfa + N_2) / 2, 
                   np.where(color_map == R, (NE_1 + NW_1) / 2, 
                       np.where(cmap_N_1 == B, N_1, (NE_N + NW_N + E_1 + W_1) / 4)))
    B_E = np.where(color_map == B, (cfa + E_2) / 2, 
                  np.where(color_map == R, (NE_1 + SE_1) / 2,
                       np.where(cmap_E_1 == B, E_1, (NE_E + SE_E + N_1 + S_1) / 4))) 
    B_S = np.where(color_map == B, (cfa + S_2) / 2,
                  np.where(color_map == R, (SE_1 + SW_1) / 2,
                       np.where(cmap_S_1 == B, S_1, (SE_S + SW_S + E_1 + W_1) / 4)))
    B_W = np.where(color_map == B, (cfa + W_2) / 2,
                  np.where(color_map == R, (NW_1 + SW_1) / 2, 
                       np.where(cmap_W_1 == B, W_1, (NW_W + SW_W + N_1 + S_1) / 4)))
    B_NE = np.where(color_map == B, (cfa + N_2 + E_2 + NE_2) / 4,
                   np.where(color_map == R, NE_1,
                       np.where(cmap_E_1 == B, (E_1 + NE_N) / 2, (N_1 + NE_E) / 2)))
    B_SE = np.where(color_map == B, (cfa + E_2 + S_2 + SE_2) / 4,
                   np.where(color_map == R, SE_1,
                       np.where(cmap_E_1 == B, (E_1 + SE_S) / 2, (S_1 + SE_E) / 2)))
    B_NW = np.where(color_map == B, (cfa + N_2 + W_2 + NW_2) / 4,
                   np.where(color_map == R, NW_1,
                       np.where(cmap_W_1 == B, (W_1 + NW_N) / 2, (N_1 + NW_W) / 2)))
    B_SW = np.where(color_map == B, (cfa + S_2 + W_2 + SW_2) / 4,
                   np.where(color_map == R, SW_1,
                       np.where(cmap_W_1 == B, (W_1 + SW_S) / 2, (S_1 + SW_W) / 2)))
    
    # calc diffs for directions
    N_2, E_2, S_2, W_2, NE_1, NE_2, SE_1, SE_2, NW_1, NW_2, SW_1, SW_2 = np.abs(cfa -
                                 np.stack((N_2, E_2, S_2, W_2, NE_1, NE_2, SE_1, SE_2, NW_1, NW_2, SW_1, SW_2)))
    # for red/blue center: 2 diffs with step 1 for green and 1 diff with step 2 for red/blue
    padded_NE_1 = np.pad(NE_1, ((1, 1), (1, 1)), 'constant', constant_values=0)
    NE_1 = (NE_1 + padded_NE_1[:-2, 2:]) / 2
    NE_1 = np.where(color_map == G, NE_1, NE_2)
    padded_SE_1 = np.pad(SE_1, ((1, 1), (1, 1)), 'constant', constant_values=0)
    SE_1 = (SE_1 + padded_SE_1[2:, 2:]) / 2
    SE_1 = np.where(color_map == G, SE_1, SE_2)
    padded_NW_1 = np.pad(NW_1, ((1, 1), (1, 1)), 'constant', constant_values=0)
    NW_1 = (NW_1 + padded_NW_1[:-2, :-2]) / 2
    NW_1 = np.where(color_map == G, NW_1, NW_2)
    padded_SW_1 = np.pad(SW_1, ((1, 1), (1, 1)), 'constant', constant_values=0)
    SW_1 = (SW_1 + padded_SW_1[2:, :-2]) / 2
    SW_1 = np.where(color_map == G, SW_1, SW_2)
    # now some diffs contain not valid values, but we won't use them further

    # calc gradients using convs on diffs
    # OK, that also can be done without convs, but seems overkill
    N_filter = np.array([[0, 0, 0, 0, 0],
                         [0, 0, 0, 0, 0],
                         [0, 0.5, 1, 0.5, 0],
                         [0, 0.5, 1, 0.5, 0],
                         [0, 0, 0, 0, 0]])
    N_2 = signal.convolve2d(N_2, N_filter, mode='valid')

    E_filter = np.array([[0, 0, 0, 0, 0],
                         [0, 0.5, 0.5, 0, 0],
                         [0, 1, 1, 0, 0],
                         [0, 0.5, 0.5, 0, 0],
                         [0, 0, 0, 0, 0]])
    E_2 = signal.convolve2d(E_2, E_filter, mode='valid')

    S_filter = np.array([[0, 0, 0, 0, 0],
                         [0, 0.5, 1, 0.5, 0],
                         [0, 0.5, 1, 0.5, 0],
                         [0, 0, 0, 0, 0],
                         [0, 0, 0, 0, 0]])
    S_2 = signal.convolve2d(S_2, S_filter, mode='valid')

    W_filter = np.array([[0, 0, 0, 0, 0],
                         [0, 0, 0.5, 0.5, 0],
                         [0, 0, 1, 1, 0],
                         [0, 0, 0.5, 0.5, 0],
                         [0, 0, 0, 0, 0]])
    W_2 = signal.convolve2d(W_2, W_filter, mode='valid')

    NE_filter = np.array([[0, 0, 0, 0, 0],
                          [0, 0, 0, 0, 0],
                          [0, 1, 1, 0, 0],
                          [0, 1, 1, 0, 0],
                          [0, 0, 0, 0, 0]])
    NE_1 = signal.convolve2d(NE_1, NE_filter, mode='valid')
    NE_2 = signal.convolve2d(NE_2, NE_filter, mode='valid')

    SE_filter = np.array([[0, 0, 0, 0, 0],
                          [0, 1, 1, 0, 0],
                          [0, 1, 1, 0, 0],
                          [0, 0, 0, 0, 0],
                          [0, 0, 0, 0, 0]])
    SE_1 = signal.convolve2d(SE_1, SE_filter, mode='valid')
    SE_2 = signal.convolve2d(SE_2, SE_filter, mode='valid')

    NW_filter = np.array([[0, 0, 0, 0, 0],
                          [0, 0, 0, 0, 0],
                          [0, 0, 1, 1, 0],
                          [0, 0, 1, 1, 0],
                          [0, 0, 0, 0, 0]])
    NW_1 = signal.convolve2d(NW_1, NW_filter, mode='valid')
    NW_2 = signal.convolve2d(NW_2, NW_filter, mode='valid')

    SW_filter = np.array([[0, 0, 0, 0, 0],
                          [0, 0, 1, 1, 0],
                          [0, 0, 1, 1, 0],
                          [0, 0, 0, 0, 0],
                          [0, 0, 0, 0, 0]])
    SW_1 = signal.convolve2d(SW_1, SW_filter, mode='valid')
    SW_2 = signal.convolve2d(SW_2, SW_filter, mode='valid')
    return ((R_N, R_E, R_S, R_W, R_NE, R_SE, R_NW, R_SW), 
            (G_N, G_E, G_S, G_W, G_NE, G_SE, G_NW, G_SW), 
            (B_N, B_E, B_S, B_W, B_NE, B_SE, B_NW, B_SW), 
            (N_2,E_2,S_2,W_2,NE_1,NE_2,SE_1,SE_2,NW_1,NW_2,SW_1,SW_2))

#### Шаг 3. Интреполируем неизвестные интенсивности

In [ ]:
def interpolate(cfa: np.array, rgb_cfa: np.array):
    # calc color map
    BAYER_FILTER = np.apply_along_axis(get_color, -1, rgb_cfa[:2, :2])
    cfa_x, cfa_y = cfa.shape
    bayer_x, bayer_y = BAYER_FILTER.shape
    reps = math.ceil(cfa_x / bayer_x), math.ceil(cfa_y / bayer_y)
    color_map = np.tile(BAYER_FILTER, reps)[:cfa_x, :cfa_y]
    
    # calc directions and grads 
    R, G, B, grads = calc_directions_and_grads(cfa, color_map)
    
    